<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></span><ul class="toc-item"><li><span><a href="#Software-and-conventions" data-toc-modified-id="Software-and-conventions-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Software and conventions</a></span></li><li><span><a href="#Compiling-WRF-Hydro" data-toc-modified-id="Compiling-WRF-Hydro-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Compiling WRF-Hydro</a></span></li></ul></li><li><span><a href="#Experiment-with-Terrain-Routing-Options" data-toc-modified-id="Experiment-with-Terrain-Routing-Options-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Experiment with Terrain Routing Options</a></span><ul class="toc-item"><li><span><a href="#Background" data-toc-modified-id="Background-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Background</a></span></li><li><span><a href="#Objective" data-toc-modified-id="Objective-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Objective</a></span></li><li><span><a href="#Step-1:-Create-and-run-first-baseline-simulation" data-toc-modified-id="Step-1:-Create-and-run-first-baseline-simulation-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Step 1: Create and run first baseline simulation</a></span></li><li><span><a href="#Step-2:-Create-and-run-the-&quot;overland-routing-off&quot;-simulation" data-toc-modified-id="Step-2:-Create-and-run-the-&quot;overland-routing-off&quot;-simulation-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Step 2: Create and run the "overland routing off" simulation</a></span></li><li><span><a href="#Step-3:-Create-and-run-the-&quot;no-terrain-routing&quot;-simulation" data-toc-modified-id="Step-3:-Create-and-run-the-&quot;no-terrain-routing&quot;-simulation-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Step 3: Create and run the "no terrain routing" simulation</a></span></li><li><span><a href="#Results" data-toc-modified-id="Results-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Results</a></span></li></ul></li><li><span><a href="#Experiment-with-Modified-Parameters" data-toc-modified-id="Experiment-with-Modified-Parameters-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Experiment with Modified Parameters</a></span><ul class="toc-item"><li><span><a href="#Step-1:-Create-a-new-template-directory-and-and-run-default-parameter-test-case" data-toc-modified-id="Step-1:-Create-a-new-template-directory-and-and-run-default-parameter-test-case-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Step 1: Create a new template directory and and run default parameter test case</a></span></li><li><span><a href="#Step-2:-Modify-NoahMP-parameters-using-NCO-tools" data-toc-modified-id="Step-2:-Modify-NoahMP-parameters-using-NCO-tools-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Step 2: Modify NoahMP parameters using NCO tools</a></span></li><li><span><a href="#Results" data-toc-modified-id="Results-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Results</a></span></li><li><span><a href="#Results" data-toc-modified-id="Results-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Results</a></span></li></ul></li></ul></div>

# Lesson 5 - Land Surface Experiments
## Overview
In Lessons 4 we experimented with different initial and boundary conditions (precipitation). In this lesson, we will experiment with different terrain physics options to evaluate impacts on streamflow. [Click here](https://ral.ucar.edu/sites/default/files/public/TerraingRouting_GroundWater.pdf) for reference material on terrain routing and groundwater in WRF-Hydro. 

**NOTE: If you have not completed Lessons 1 through 4, please stop and do so now.**

### Software and conventions
The easiest way to run these lessons is via the [wrfhydro/training](https://hub.docker.com/r/wrfhydro/training/) Docker container, which has all software dependencies and data pre-installed.

For a complete description of the software environment used for this training please see [Getting started](http://localhost:8888/notebooks/wrf-hydro-training/lessons/Lesson-0-start.ipynb).

You may either execute commands by running each cell of this notebook. Alternatively, you may open a terminal in Jupyter Lab by selecting `New -> Terminal` in your `Home` tab of Jupyter Lab and input the commands manually if you prefer. You can also use your own terminal by logging into the container with the command `docker exec -it wrf-hydro-training bash`

All paths used in this lesson assume that the lesson materials are located under your home directory in a folder named `wrf-hydro-training`. If your materials are located in another directory, you will not be able to run the commands in this notebook inside Jupyter and will need to type them manually in your terminal session. 

### Compiling WRF-Hydro
Below are the commands to compile WRF-Hydro. We are doing a quick short-cut to edit the setEnvar.sh to make sure HYDRO_D and SPATIAL_SOIL are both active. See [Lesson 1](http://localhost:8888/notebooks/wrf-hydro-training/lessons/Lesson-1-compile.ipynb) if you have any questions on these settings.

**NOTE: You only need to do these steps if you do not already have a compiled binary. If you do, you can skip the compile steps.**

In [ ]:
%%bash
# Change to the trunk/NDHMS directory and configure for gfort
cd ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/; ./configure 2

# Make a copy of the template environment variable file, setEnvars.sh
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/template/setEnvar.sh \
~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/setEnvar.sh

# Edit setEnvar.sh to turn on debug (HYDRO_D=1) and spatial soils (SPATIAL_SOIL=1)
sed -i -e 's#export HYDRO_D=0#export HYDRO_D=1#g' ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/setEnvar.sh
sed -i -e 's#export SPATIAL_SOIL=0#export SPATIAL_SOIL=1#g' ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/setEnvar.sh

# Compile the code
./compile_offline_NoahMP.sh setEnvar.sh >> compile.log 2>&1

# Check to make sure it completed successfully and confirm the correct compile-time options were set.
tail -13 ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/compile.log

## Experiment with Terrain Routing Options
### Background
In most global and regional land surface models, model cells are simulated as independent columns and "runoff" is interpreted as excess surface and subsurface water that is "scraped" from each cell after vertical infiltration and drainage processes complete. WRF-Hydro adds functionality to allow this excess water to instead move laterally around the land surface based on topographic and head gradients. WRF-Hydro can explicitly represent overland flow processes, where infiltration and saturation excess water propagates over the surface through a diffusive wave formulation. In addition, WRF-Hydro includes shallow subsurface flow, allowing saturated soil water to move from cell to cell through a Boussinesq approximation. For more detailed information refer to [WRF-Hydro V5 Technical Description](https://ral.ucar.edu/sites/default/files/public/WRF-HydroV5TechnicalDescription.pdf). 

### Objective
Experiment with lateral flow physics options, then compare hydrographs with and without terrain routing processes active.

### Step 1: Create and run first baseline simulation
As in Lesson 4, we will create a template directory to use for the terrain physics experiments.

**Step 1a: Create a template run directory**

In [ ]:
%%bash
# Make a new directory for our baseline simulation
mkdir -p ~/wrf-hydro-training/output/lesson5/run_gridded_template

# Copy our model files to the simulation directory
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run/*.TBL \
~/wrf-hydro-training/output/lesson5/run_gridded_template
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run/wrf_hydro.exe \
~/wrf-hydro-training/output/lesson5/run_gridded_template
cp ~/wrf-hydro-training/example_case/Gridded/namelist.hrldas \
~/wrf-hydro-training/output/lesson5/run_gridded_template
cp ~/wrf-hydro-training/example_case/Gridded/hydro.namelist \
~/wrf-hydro-training/output/lesson5/run_gridded_template

# Create symbolic links to large domain files
cp -as $HOME/wrf-hydro-training/example_case/FORCING \
~/wrf-hydro-training/output/lesson5/run_gridded_template
cp -as $HOME/wrf-hydro-training/example_case/Gridded/RESTART \
~/wrf-hydro-training/output/lesson5/run_gridded_template
cp -as $HOME/wrf-hydro-training/example_case/Gridded/DOMAIN \
~/wrf-hydro-training/output/lesson5/run_gridded_template


**Step 1b: Now make a copy of the template directory and run the baseline simulation**

First, we remind ourselves what we are running as a baseline. Note in particular that `OVRTSWCRT=1` and `SUBRTSWCRT=1`, meaning that both overland and subsurface flow routing options are active.

In [ ]:
%%bash
# Make a copy of the template directory for the baseline run
cp -r ~/wrf-hydro-training/output/lesson5/run_gridded_template \
~/wrf-hydro-training/output/lesson5/run_gridded_baseline
# View the hydro.namelist settings
cat ~/wrf-hydro-training/output/lesson5/run_gridded_baseline/hydro.namelist

Now run the baseline model simulation as-is.

In [ ]:
%%bash
# Run the simulation
cd ~/wrf-hydro-training/output/lesson5/run_gridded_baseline
mpirun -np 2 ./wrf_hydro.exe >> run.log 2>&1

Check that the simulation finished successfully

In [ ]:
%%bash
tail -1 ~/wrf-hydro-training/output/lesson5/run_gridded_baseline/diag_hydro.00000

### Step 2: Create and run the "overland routing off" simulation
We will run the same experiment as the above baseline run, but with the overland routing module turned off.

**Step 2a: Create a new run directory**

We first create a `run_overland_routing_off` directory from the template.

In [ ]:
%%bash
cp -r ~/wrf-hydro-training/output/lesson5/run_gridded_template \
~/wrf-hydro-training/output/lesson5/run_overland_routing_off

We will make one modification to `hydro.namelist` to turn off the overland routing modules. 

**Step 2b: Edit the `hydro.namelist` file**

For this experiment, we will set the *OVRTSWCRT* physics option to 0, which deactivates overland terrain routing modules. 

```
! Switch to activate surface overland flow routing...(0=no, 1=yes)
OVRTSWCRT = 0
```

**Step 2c: Run the simulation and make sure it finishes successfully**

In [ ]:
%%bash
cd ~/wrf-hydro-training/output/lesson5/run_overland_routing_off
mpirun -np 2 ./wrf_hydro.exe >> run.log 2>&1

Check that the simulation finished successfully

In [ ]:
%%bash
tail -1 ~/wrf-hydro-training/output/lesson5/run_overland_routing_off/diag_hydro.00000

### Step 3: Create and run the "no terrain routing" simulation
Now we run an experiment with all terrain routing options turned off. This will approximate a standard column physics configuration.

**Step 3a: Create a new run directory**

We first create a `run_no_terrain_routing` directory from the template.

In [ ]:
%%bash
cp -r ~/wrf-hydro-training/output/lesson5/run_gridded_template \
~/wrf-hydro-training/output/lesson5/run_no_terrain_routing

**Step3b: Edit the `hydro.namelist` file**
We will make three modifications to `hydro.namelist` to turn off all of the terrain routing modules. 

For this experiment, we will set the *OVRTSWCRT* and *SUBRTSWCRT* physics options to 0, which deactivates both overland and subsurface terrain routing modules. 

```
! Switch to activate subsurface routing...(0=no, 1=yes)
SUBRTSWCRT = 0
```
```
! Switch to activate surface overland flow routing...(0=no, 1=yes)
OVRTSWCRT = 0
```

Since we are deactivating all terrain routing physics, we also need to tell the model not to overwrite initial conditions with the routing grids using the *rst_typ* option. Deactivate this function by setting *rst_typ* to 0.

```
! Reset the LSM soil states from the high-res routing restart file (1=overwrite, 0=no overwrite)
! NOTE: Only turn this option on if overland or subsurface rotuing is active!
rst_typ = 0
```

**Step 3c: Run the simulation**

In [ ]:
%%bash
cd ~/wrf-hydro-training/output/lesson5/run_no_terrain_routing
mpirun -np 2 ./wrf_hydro.exe >> run.log 2>&1

Check that the simulation finished successfully

In [ ]:
%%bash
tail -1 ~/wrf-hydro-training/output/lesson5/run_no_terrain_routing/diag_hydro.00000

### Results
We will now look at the differences in streamflow between our baseline run with all terrain routing options on and our experiments with different terrain routing options turned off.

We will use Python and the `xarray` library to load the data and plot hydrographs. For an intro to these tools, please see Lesson 3.

**Load the xarray python package**

In [ ]:
# Load the xarray package
%matplotlib inline
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd

**Load the CHANOBS streamflow datasets**

We are going to use the CHANOBS files (vs. CHRTOUT files) because it will limit the number of reaches to only those which have a gage.

In [ ]:
chanobs_control = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson5/run_gridded_baseline/*CHANOBS*',
                            combine='by_coords')
chanobs_ov_off = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson5/run_overland_routing_off/*CHANOBS*',
                            combine='by_coords')
chanobs_terrain_off = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson5/run_no_terrain_routing/*CHANOBS*',
                            combine='by_coords')
obs = pd.read_csv('/home/docker/wrf-hydro-training/example_case/obs_event.csv',dtype=str)
obs['dateTime'] = pd.to_datetime(obs['DATE'])
obs['streamflow_cms'] = pd.to_numeric(obs['streamflow_cms'])

**Plot the hydrographs**

In [ ]:
fig, axes = plt.subplots(ncols=1,figsize=(12, 6))
plt.suptitle('Hydrographs for terrain routing physics options',fontsize=24)
chanobs_control.sel(feature_id = 3).streamflow.plot(label='All Terrain Routing On',
                                                        color='black',
                                                        linestyle='--')
chanobs_ov_off.sel(feature_id = 3).streamflow.plot(label='Overland Routing Off',
                                                        color='blue',
                                                        linestyle='-')
chanobs_terrain_off.sel(feature_id = 3).streamflow.plot(label='No Terrain Routing',
                                                        color='red',
                                                        linestyle='-')

obs[obs['Location'] == 'Desague'].plot(x='dateTime',
                                       y='streamflow_cms',
                                       ax=axes,
                                       label='Observed',
                                       color='grey')
plt.ylim(0,16)
plt.legend()
plt.show()


**Load the LDASOUT land model output files**

In our simulations, we specified land surface model output (LDASOUT) files every hour. We will read those in using xarray.

In [ ]:
ldasout_terrain_on = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson5/run_gridded_baseline/*.LDASOUT*',
                            combine='by_coords')
ldasout_terrain_off = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson5/run_no_terrain_routing/*.LDASOUT*',
                            combine='by_coords')

**Plot the soil moisture states**

Among other variables, the LDASOUT files include soil moisture states (*SOIL_M*) in each of the 4 soil layers. We will select and plot soil moisture in the top soil layer from each simulation at 2011-08-29 00Z. For reference, we also plot the soil types from the model input geogrid file.

In [ ]:
# Select data for 2017-11-16 12Z at surface layer
ldasout_terrain_on_time = ldasout_terrain_on.sel(time = '2017-11-16T12:00:00').sel(soil_layers_stag = 0)
ldasout_terrain_off_time = ldasout_terrain_off.sel(time = '2017-11-16T12:00:00').sel(soil_layers_stag = 0)

# Plot the top layer soil moisture states
fig, axes = plt.subplots(ncols=2,figsize=(12, 6))
plt.suptitle('Surface soil moisture states for terrain routing on and off',fontsize=24)
ldasout_terrain_on_time.SOIL_M.plot(ax=axes[0],vmin=0.30,vmax=0.45,cmap='viridis_r')
axes[0].set_title('Terrain routing on')
ldasout_terrain_off_time.SOIL_M.plot(ax=axes[1],vmin=0.30,vmax=0.45,cmap='viridis_r')
axes[1].set_title('Terrain routing off')
plt.show()

In [ ]:
# Load the geogrid dataset for reference
geogrid = xr.open_dataset('~/wrf-hydro-training/output/lesson5/run_gridded_baseline/DOMAIN/geo_em.d01.nc')

In [ ]:
# Plot the dominant soil type
fig, axes = plt.subplots(figsize=(6, 6))
geogrid.SCT_DOM.plot(cmap="tab20")
axes.set_title('Soil Type')
plt.show()

The top-level soil moisture state shows significant differences with and without terrain routing active. Overall, we see much more heterogeneity in the "terrain routing on" run (above, left) due to lateral redistribution of water. In the "terrain routing off" run (above, right), soil moisture distribution is largely controlled by vertical soil properties (e.g., saturated hydraulic conductivity, porosity), and you distinctly see the soil type pattern in the soil moisture states.

**Plot the mean soil moisture time series**

In [ ]:
# Calculate the mean bottom-layer soil moisture across the domain
smois_terron_avg = ldasout_terrain_on.SOIL_M.sel(soil_layers_stag = 3).mean(dim=('y','x'))
smois_terroff_avg = ldasout_terrain_off.SOIL_M.sel(soil_layers_stag = 3).mean(dim=('y','x')) 

# Plot the soil moisture time series
fig, axes = plt.subplots(ncols=1,figsize=(12, 6))
plt.suptitle('Average Soil Moisture: Bottom Layer',fontsize=24)
smois_terron_avg.plot(label='Terrain Routing On', color='black', linestyle='-')
smois_terroff_avg.plot(label='Terrain Routing Off', color='red', linestyle='-')
plt.legend()
plt.show()

**Load the GWOUT groundwater bucket model output files**

In [ ]:
gwbucket_terrain_on = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson5/run_gridded_baseline/*GWOUT*',
                            combine='by_coords')
gwbucket_terrain_off = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson5/run_no_terrain_routing/*GWOUT*',
                            combine='by_coords')
gwbucket_ov_off = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson5/run_overland_routing_off/*GWOUT*',
                            combine='by_coords')

**Plot the mean groundwater bucket level time series**

Note that groundwater bucket "basins" vary in size, so a simple mean is not a good way to track mass. We do it here for simplicity and to get a general sense of the bucket behavior across the domain.

In [ ]:
# Calculate the mean bucket level across the domain
gwlevel_terron_avg = gwbucket_terrain_on.depth.mean(dim=('feature_id'))
gwlevel_terroff_avg = gwbucket_terrain_off.depth.mean(dim=('feature_id')) 

# Plot the bucket level time series
fig, axes = plt.subplots(ncols=1,figsize=(12, 6))
plt.suptitle('Average Groundwater Bucket Level',fontsize=24)
gwlevel_terron_avg.plot(label='Terrain Routing On', color='black', linestyle='-')
gwlevel_terroff_avg.plot(label='Terrain Routing Off', color='red', linestyle='-')
plt.legend()
plt.show()

**IMPORTANT NOTE**: This particular test case has been calibrated to the full terrain routing physics options, so will obviously be sub-optimal for a different physics configuration. You should adjust your parameters to your particular model configuration. You should also re-spinup the model to give it time to adjust to the new physics. We have done neither of those steps in this short example.

## Experiment with Modified Parameters
There are a number of key parameters that impact water partitioning, storage, and movement through the model system. We have pulled many of the most important model parameters into NetCDF files to ease parameter display and manipulation, as well as to allow the parameters to vary independently in space. Key terrain routing parameter files include:
* soil_properties.nc - NoahMP soil and vegetation properties (LSM grid)
* hydro2dtbl.nc - Lateral routing model soil and surface parameters (LSM grid)
* Fulldom_hires.nc - Lateral routing model high-res parameters (routing grid)
* GWBUCKPARM.nc - Groundwater baseflow bucket model parameters (groundwater basin objects)

In this lesson, we will manipulate parameters in the `soil_properties.nc` (*refkdt*, *dksat*, *bexp*, *slope*) file.

### Step 1: Create a new template directory and and run default parameter test case
As in the first section, we will make a new simulation directory and use this as a template for creating multiple new simulation directories.

**Step 1a: Create a new template directory for the parameter experiments.**

In [ ]:
%%bash
# Make a new directory for our default baseline simulation
mkdir -p ~/wrf-hydro-training/output/lesson5/run_default_template

# Copy our model files to the simulation directory
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run/*.TBL \
~/wrf-hydro-training/output/lesson5/run_default_template
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run/wrf_hydro.exe \
~/wrf-hydro-training/output/lesson5/run_default_template
cp ~/wrf-hydro-training/output/lesson5/run_gridded_template/namelist.hrldas \
~/wrf-hydro-training/output/lesson5/run_default_template
cp ~/wrf-hydro-training/output/lesson5/run_gridded_template/hydro.namelist \
~/wrf-hydro-training/output/lesson5/run_default_template

# Create symbolic links to large domain files
cp -as $HOME/wrf-hydro-training/example_case/FORCING \
~/wrf-hydro-training/output/lesson5/run_default_template
cp -as $HOME/wrf-hydro-training/example_case/Gridded/RESTART \
~/wrf-hydro-training/output/lesson5/run_default_template
cp -as $HOME/wrf-hydro-training/example_case/Gridded/DOMAIN \
~/wrf-hydro-training/output/lesson5/run_default_template


**Step 1b: Setup and run the default parameter baseline experiment**

We now make a new run directory to run the baseline simulation with default (uncalibrated) parameters.

In [ ]:
%%bash
cp -r ~/wrf-hydro-training/output/lesson5/run_default_template \
~/wrf-hydro-training/output/lesson5/run_parameter_baseline

Launch the baseline run and make sure it completes successfully.

In [ ]:
%%bash
cd ~/wrf-hydro-training/output/lesson5/run_parameter_baseline
mpirun -np 2 ./wrf_hydro.exe >> run.log 2>&1

In [ ]:
%%bash
tail -1 ~/wrf-hydro-training/output/lesson5/run_parameter_baseline/diag_hydro.00000

**Step 1c: Plot the hydrograph for the baseline (default) run**

We want to take a quick look at the hydrograph for the default parameter run so we see what behavior we might want to adjust.

In [ ]:
# Load the xarray package
%matplotlib inline
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd

# Pull the channel output files into xarray objects 
chanobs_control = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson5/run_parameter_baseline/*CHANOBS*',
                            combine='by_coords')
obs = pd.read_csv('/home/docker/wrf-hydro-training/example_case/obs_event.csv',dtype=str)
obs['dateTime'] = pd.to_datetime(obs['DATE'])
obs['streamflow_cms'] = pd.to_numeric(obs['streamflow_cms'])

# Plot the baseline hydrograph
fig, axes = plt.subplots(ncols=1,figsize=(12, 6))
plt.suptitle('Hydrographs for parameter experiment',fontsize=24)
chanobs_control.sel(feature_id = 3).streamflow.plot(label='Control',
                                                        color='black',
                                                        linestyle='--')
obs[obs['Location'] == 'Desague'].plot(x='dateTime',
                                       y='streamflow_cms',
                                       ax=axes,
                                       label='Observed',
                                       color='grey')
plt.ylim(0,30)
plt.legend()
plt.show()

### Step 2: Modify NoahMP parameters using NCO tools

We will create a new simulation directory for our parameter manipulation experiment. All of the parameter file edits will be done using NCO. NCO (NetCDF Operators, http://nco.sourceforge.net/) is a set of useful utilities to manipulate NetCDF files.

**Step 2a: Setup the parameter experiment run directory**

First make a new run directory where we can start modifying parameter files.

In [ ]:
%%bash
cp -r ~/wrf-hydro-training/output/lesson5/run_default_template \
~/wrf-hydro-training/output/lesson5/run_parameter_mods

**Step 2b: Use the NCO command `ncap2` to modify the *refkdt* parameter values in the `soil_properties.nc` file.**

**REFKDT**

There are a number of model parameters that affect lateral terrain flow. One important Noah/NoahMP parameter that we commonly adjust when activating terrain routing is *refkdt*. *Refkdt* controls how easily precipitation reaching the surface infiltrates into the soil column vs. stays on the surface where it will be "scraped" off as surface runoff. Higher values of *refkdt* lead to more infiltration and less surface (fast) runoff. This tunable parameter can be set to a relatively high value (e.g., 3.0) suitable for running the column land surface model only. When activating terrain routing to explicitly model these processes, we often reduce this parameter. In addition, if you are calling the land surface model on a small timestep (e.g., seconds to minutes), you may want to reduce this parameter to compensate for the more frequent calls to the vertical infiltration scheme. 

Here we will use the NCO command `ncap2` to decrease the *refkdt* value.

First, we check the current parameter values using `ncdump`.

In [ ]:
%%bash
ncdump -v refkdt ~/wrf-hydro-training/output/lesson5/run_parameter_mods/DOMAIN/soil_properties.nc \
| tail -n 10

The default value for *refkdt* is a global 3.0. We modify the parameter values using `ncap2` to a lower value (0.1) to encourage less infiltration and more flow available for exfiltration to the channel.

In [ ]:
%%bash
ncap2 -O -s "refkdt=refkdt*0.0+0.1" \
~/wrf-hydro-training/output/lesson5/run_parameter_mods/DOMAIN/soil_properties.nc \
~/wrf-hydro-training/output/lesson5/run_parameter_mods/DOMAIN/soil_properties.nc

We check to make sure the changes are as expected.

In [ ]:
%%bash
ncdump -v refkdt ~/wrf-hydro-training/output/lesson5/run_parameter_mods/DOMAIN/soil_properties.nc \
| tail -n 10

**Step 3b: Run the simulation**

Now we are ready to run a simulation with our new modified parameters.

In [ ]:
%%bash
cd ~/wrf-hydro-training/output/lesson5/run_parameter_mods
mpirun -np 2 ./wrf_hydro.exe >> run.log 2>&1

Check to make sure your run completed successfully.

In [ ]:
%%bash
tail -1 ~/wrf-hydro-training/output/lesson5/run_parameter_mods/diag_hydro.00000

### Results
We will now look at the differences in streamflow between our baseline run with default parmeters and the simulation using our new modified parameters.

We will use Python and the `xarray` library to load the data and plot hydrographs. For an intro to these tools, please see Lesson 3.

In [ ]:
chanobs_control = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson5/run_parameter_baseline/*CHANOBS*',
                            combine='by_coords')
chanobs_param_mods = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson5/run_parameter_mods/*CHANOBS*',
                            combine='by_coords')

In [ ]:
fig, axes = plt.subplots(ncols=1,figsize=(12, 6))
plt.suptitle('Hydrographs for parameter experiment',fontsize=24)
chanobs_control.sel(feature_id = 3).streamflow.plot(label='Control, refkdt=3',
                                                        color='black',
                                                        linestyle='--')
chanobs_param_mods.sel(feature_id = 3).streamflow.plot(label='refkdt=0.1',
                                                        color='blue',
                                                        linestyle='-')
obs[obs['Location'] == 'Desague'].plot(x='dateTime',
                                       y='streamflow_cms',
                                       ax=axes,
                                       label='Observed',
                                       color='grey')
plt.ylim(0,15)
plt.legend()
plt.show()


**Step 2c: Use the NCO command ncap2 to modify the bexp parameter values in the soil_properties.nc file.**

**BEXP**

The bexp parameter controls how actual conductivity is scaled from saturated conductivity based on soil water content. Initial values are estimated based on soil texture class, but reported ranges have large (many orders of magnitude) variability.

The equation for the application of the BEXP parameter in the calculation of unsaturated hydaulic conductivity (WCND) the NoahMP code is as follows:

 WCND  = parameters%DKSAT(ISOIL) * FACTR2 ** EXPON
 where: 
   parameters%DKSAT(ISOIL) is the saturated hydraulic conductivity (DKSAT) value
   FACTR2 is fraction of actual to saturated soil water content [MAX(0.01, SMC/parameters%SMCMAX(ISOIL))]   
   EXPON is 2.0*parameters%BEXP(ISOIL) + 3.0

First we make a new directory and repeat our previous change to refkdt so that we are building on the previous experiment.


In [ ]:
%%bash
cp -r ~/wrf-hydro-training/output/lesson5/run_default_template \
~/wrf-hydro-training/output/lesson5/run_parameter_mods2

ncap2 -O -s "refkdt=refkdt*0.0+0.1" \
~/wrf-hydro-training/output/lesson5/run_parameter_mods2/DOMAIN/soil_properties.nc \
~/wrf-hydro-training/output/lesson5/run_parameter_mods2/DOMAIN/soil_properties.nc


First, we check the current parameter values using `ncdump`. Note that these also vary spatially by soil type.

In [ ]:
%%bash
ncdump -v bexp ~/wrf-hydro-training/output/lesson5/run_parameter_mods2/DOMAIN/soil_properties.nc \
| tail -n 10

Here we will use the NCO command `ncap2` to decrease *bexp* by a factor of 0.5 to promote more runoff.

In [ ]:
%%bash
ncap2 -O -s "bexp=bexp*0.5" \
~/wrf-hydro-training/output/lesson5/run_parameter_mods2/DOMAIN/soil_properties.nc \
~/wrf-hydro-training/output/lesson5/run_parameter_mods2/DOMAIN/soil_properties.nc
ncdump -v bexp ~/wrf-hydro-training/output/lesson5/run_parameter_mods2/DOMAIN/soil_properties.nc \
| tail -n 10

Now we will run the simulation with the decreased *bexp* and then plot the hydrographs.

In [ ]:
%%bash
cd ~/wrf-hydro-training/output/lesson5/run_parameter_mods2
mpirun -np 2 ./wrf_hydro.exe >> run.log 2>&1

In [ ]:
chanobs_param_mods2 = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson5/run_parameter_mods2/*CHANOBS*',
                            combine='by_coords')

In [ ]:
fig, axes = plt.subplots(ncols=1,figsize=(12, 6))
plt.suptitle('Hydrographs for parameter experiment',fontsize=24)
chanobs_control.sel(feature_id = 3).streamflow.plot(label='Control, w/refkdt=3',
                                                        color='black',
                                                        linestyle='--')
chanobs_param_mods.sel(feature_id = 3).streamflow.plot(label='refkdt=0.1',
                                                        color='blue',
                                                        linestyle='-')
chanobs_param_mods2.sel(feature_id = 3).streamflow.plot(label='refkdt=0.1, bexp*0.5',
                                                        color='green',
                                                        linestyle='-')
obs[obs['Location'] == 'Desague'].plot(x='dateTime',
                                       y='streamflow_cms',
                                       ax=axes,
                                       label='Observed',
                                       color='grey')
plt.ylim(0,20)
plt.legend()
plt.show()

**Step 2d: Use the NCO command `ncap2` to modify the *dksat* parameter values in the `soil_properties.nc` file.**

**dksat**

As with most physically based hydrological models, the soil saturated hydraulic conductivity, dksat, controls the speed at which water moves through the subsurface.  This is a sensitive parameter in the model, and while easy to measure at the point scale, dksat is tricky to estimate at the scale of kilometers.  This is a common calibration parameter.  

First we make a new directory and modify `refkdt` and `bexp` to the same values as the previous experiments so that we are building on our previous simulations.

In [ ]:
%%bash
cp -r ~/wrf-hydro-training/output/lesson5/run_default_template \
~/wrf-hydro-training/output/lesson5/run_parameter_mods3

ncap2 -O -s "refkdt=refkdt*0.0+0.1" \
~/wrf-hydro-training/output/lesson5/run_parameter_mods3/DOMAIN/soil_properties.nc \
~/wrf-hydro-training/output/lesson5/run_parameter_mods3/DOMAIN/soil_properties.nc

ncap2 -O -s "bexp=bexp*0.5" \
~/wrf-hydro-training/output/lesson5/run_parameter_mods3/DOMAIN/soil_properties.nc \
~/wrf-hydro-training/output/lesson5/run_parameter_mods3/DOMAIN/soil_properties.nc

Here we will view the default dksat and then reduce dksat by a factor of 0.5 using the ncap2 NCO command.

In [ ]:
%%bash
ncdump -v dksat ~/wrf-hydro-training/output/lesson5/run_parameter_mods3/DOMAIN/soil_properties.nc \
| tail -n 10

In [ ]:
%%bash
ncap2 -O -s "dksat=dksat*0.5" \
~/wrf-hydro-training/output/lesson5/run_parameter_mods3/DOMAIN/soil_properties.nc \
~/wrf-hydro-training/output/lesson5/run_parameter_mods3/DOMAIN/soil_properties.nc
ncdump -v dksat ~/wrf-hydro-training/output/lesson5/run_parameter_mods3/DOMAIN/soil_properties.nc \
| tail -n 10

**Step 2e: Use the NCO command `ncap2` to modify the *slope* parameter values in the `soil_properties.nc` file.**

**SLOPE**

Another important Noah/NoahMP parameter that we commonly adjust is *slope*. Originally estimated based on land surface topography (hence the name *slope*), the *slope* parameter actually controls how open or closed the bottom boundary of the soil column is. Values range from 0 to 1, where 0 is a completely closed bottom boundary and 1 is completely open. Lower *slope* values will keep more water in the soil column, while higher values will allow more water to drain to the channel or to deeper baseflow stores, depending on the selected baseflow physics options.

As with *refkdt* and *dksat* above, we will use the NCO command `ncap2` to modify the *slope* value.

First, we check the current parameter values using `ncdump`.

In [ ]:
%%bash
ncdump -v slope ~/wrf-hydro-training/output/lesson5/run_parameter_mods3/DOMAIN/soil_properties.nc \
| tail -n 10

The default value for *slope* is a global 0.1. We modify the parameter values using `ncap2` to open the bottom boundary (slope*0+0.5) and increase the deeper baseflow component.

In [ ]:
%%bash
ncap2 -O -s "slope=slope*0+0.5" \
~/wrf-hydro-training/output/lesson5/run_parameter_mods3/DOMAIN/soil_properties.nc \
~/wrf-hydro-training/output/lesson5/run_parameter_mods3/DOMAIN/soil_properties.nc
ncdump -v slope ~/wrf-hydro-training/output/lesson5/run_parameter_mods3/DOMAIN/soil_properties.nc \
| tail -n 10

Note: we are running this in the same directory as before, but will be able to plot each of the experiments because the datasets are loaded. 

In [ ]:
%%bash
cd ~/wrf-hydro-training/output/lesson5/run_parameter_mods3
mpirun -np 2 ./wrf_hydro.exe >> run.log 2>&1

In [ ]:
chanobs_param_mods3 = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson5/run_parameter_mods3/*CHANOBS*',
                            combine='by_coords')

In [ ]:
fig, axes = plt.subplots(ncols=1,figsize=(12, 6))
plt.suptitle('Hydrographs for parameter experiment',fontsize=24)
chanobs_control.sel(feature_id = 3).streamflow.plot(label='Control, w/refkdt=3, slope=0.1',
                                                        color='black',
                                                        linestyle='--')
chanobs_param_mods.sel(feature_id = 3).streamflow.plot(label='refkdt=0.1, slope=0.1',
                                                        color='blue',
                                                        linestyle='-')
chanobs_param_mods2.sel(feature_id = 3).streamflow.plot(label='refkdt=0.1, slope=0.1, bexp*0.5',
                                                        color='green',
                                                        linestyle='-')
chanobs_param_mods3.sel(feature_id = 3).streamflow.plot(label='refkdt=0.1, bexp*0.5, slope=0.5, dksat*0.5',
                                                        color='orange',
                                                        linestyle='-')

obs[obs['Location'] == 'Desague'].plot(x='dateTime',
                                       y='streamflow_cms',
                                       ax=axes,
                                       label='Observed',
                                       color='grey')
plt.ylim(0,15)
plt.legend()
plt.show()

**OPTIONAL::: RETDEPRTFAC**

The *RETDEPRTFAC* parameter is a multiplier on the maximum retention depth. Ponded water on the surface above this retention depth threshold can be moved around the landscape via overland flow. The default value in the code is quite small (~0.001mm, though variable by terrain slope) to allow almost all ponded water to be available for routing. However, in many regions landscape features like wetlands, small detention ponds, and heavy vegetation litter/debris can trap water on the land surface. Increasing the *RETDEPRTFAC* multiplier will hold more ponded water on the surface before it becomes runoff.

We will use the NCO command `ncap2` to reduce *RETDEPRTFAC* value to 0.

First, we check the current parameter values.

In [ ]:
%%bash
cp -r ~/wrf-hydro-training/output/lesson5/run_default_template \
~/wrf-hydro-training/output/lesson5/run_parameter_mods4

ncap2 -O -s "refkdt=refkdt*0.0+0.1" \
~/wrf-hydro-training/output/lesson5/run_parameter_mods4/DOMAIN/soil_properties.nc \
~/wrf-hydro-training/output/lesson5/run_parameter_mods4/DOMAIN/soil_properties.nc

ncap2 -O -s "bexp=bexp*0.5" \
~/wrf-hydro-training/output/lesson5/run_parameter_mods4/DOMAIN/soil_properties.nc \
~/wrf-hydro-training/output/lesson5/run_parameter_mods4/DOMAIN/soil_properties.nc

ncap2 -O -s "dksat=dksat*0.5" \
~/wrf-hydro-training/output/lesson5/run_parameter_mods4/DOMAIN/soil_properties.nc \
~/wrf-hydro-training/output/lesson5/run_parameter_mods4/DOMAIN/soil_properties.nc

ncap2 -O -s "slope=slope*0+0.5" \
~/wrf-hydro-training/output/lesson5/run_parameter_mods4/DOMAIN/soil_properties.nc \
~/wrf-hydro-training/output/lesson5/run_parameter_mods4/DOMAIN/soil_properties.nc

In [ ]:
%%bash
ncdump -v RETDEPRTFAC ~/wrf-hydro-training/output/lesson5/run_parameter_mods4/DOMAIN/Fulldom_hires.nc \
| tail -n 10

Then, we modify the parameter values using `ncap2` and confirm our changes.

In [ ]:
%%bash
ncap2 -O -s "RETDEPRTFAC=RETDEPRTFAC*10000.0" \
~/wrf-hydro-training/output/lesson5/run_parameter_mods4/DOMAIN/Fulldom_hires.nc \
~/wrf-hydro-training/output/lesson5/run_parameter_mods4/DOMAIN/Fulldom_hires.nc
ncdump -v RETDEPRTFAC ~/wrf-hydro-training/output/lesson5/run_parameter_mods4/DOMAIN/Fulldom_hires.nc \
| tail -n 10

**Step 3b: Run the simulation**

Now we are ready to run a simulation with our new modified parameters.

In [ ]:
%%bash
cd ~/wrf-hydro-training/output/lesson5/run_parameter_mods4
mpirun -np 2 ./wrf_hydro.exe >> run.log 2>&1

Check to make sure your run completed successfully.

In [ ]:
%%bash
tail -1 ~/wrf-hydro-training/output/lesson5/run_parameter_mods4/diag_hydro.00000

### Results
We will now look at the differences in streamflow between our baseline run with default parmeters and the simulation using our new modified parameters.

We will use Python and the `xarray` library to load the data and plot hydrographs. For an intro to these tools, please see Lesson 3.

**Load the CHANOBS streamflow datasets and plot the results.**

We are going to use the CHANOBS files because it will limit the number of grid cells to only those which we have specified have a gage.

In [ ]:
chanobs_param_mods4 = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson5/run_parameter_mods4/*CHANOBS*',
                            combine='by_coords')

In [ ]:
fig, axes = plt.subplots(ncols=1,figsize=(12, 6))
plt.suptitle('Hydrographs for parameter experiment',fontsize=24)
chanobs_control.sel(feature_id = 3).streamflow.plot(label='Control, w/refkdt=3, slope=0.1',
                                                        color='black',
                                                        linestyle='--')
chanobs_param_mods.sel(feature_id = 3).streamflow.plot(label='refkdt=0.1, slope=0.1',
                                                        color='blue',
                                                        linestyle='-')
chanobs_param_mods2.sel(feature_id = 3).streamflow.plot(label='refkdt=0.1, slope=0.1, bexp*2',
                                                        color='green',
                                                        linestyle='-')
chanobs_param_mods3.sel(feature_id = 3).streamflow.plot(label='refkdt=0.1, slope=0.1, bexp*0.5, dksat*0.5',
                                                        color='orange',
                                                        linestyle='-')
chanobs_param_mods4.sel(feature_id = 3).streamflow.plot(label='refkdt=0.1, bexp*0.5, dksat*0.5, slope=0.5, retdeprtfac*10000',
                                                        color='red',
                                                        linestyle='-')


obs[obs['Location'] == 'Desague'].plot(x='dateTime',
                                       y='streamflow_cms',
                                       ax=axes,
                                       label='Observed',
                                       color='grey')
plt.ylim(0,20)
plt.legend()
plt.show()

# Next up - Do it yourself!
This concludes Lesson 5. Spend some time creating your own parameter and physics experiments.

**IT IS BEST TO EITHER SHUTDOWN THIS LESSON OR CLOSE IT BEFORE PROCEEDING TO THE NEXT LESSON TO AVOID POSSIBLY EXCEEDING ALLOCATED MEMORY. Shutdown the lesson be either closing the browser tab for the lesson or selecting `KERNAL->SHUTDOWN` in the jupyter notebook toolbar.**